# Lab 4

## Decision Tree

For this lab, we are going to implement a decision tree based on the C4.5 algorithm. C4.5 provides several improvements over ID3 though the base structure is very similar. C4.5 removed the restriction that features must be categorical by dynamically defining a discrete attribute (based on numerical variables) that partitions the continuous attribute value into a discrete set of intervals. C4.5 converts the trained trees (i.e. the output of the ID3 algorithm) into sets of if-then rules. 

We will start with our titanic dataset.

**Note:** Exercises can be autograded and count towards your lab and assignment score. Problems are graded for participation.

In [1]:
from pathlib import Path
home = str(Path.home()) # all other paths are relative to this path. change to something else if this is not the case on your system

In [2]:
%load_ext autoreload
%autoreload 2

# make sure your run the cell above before running this
import Lab4_helper

For developing this lab, we can use famous Titanic Kaggle dataset. Description of the data is found https://www.kaggle.com/c/titanic/data.

In [3]:
import pandas as pd
titanic_df = pd.read_csv(
    f"{home}/csc-466-student/data/titanic.csv"
)
titanic_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


We need to do some simple preprocessing before our neural network can deal with this data. 

In [4]:
features = ['Pclass','Sex','Age','SibSp','Parch','Fare','Cabin','Embarked']
titanic_df2 = titanic_df.loc[:,features]
titanic_df2['CabinLetter'] = titanic_df2['Cabin'].str.slice(0,1)
X = titanic_df2.drop('Cabin',axis=1)#.dropna()
X['CabinLetter'] = X['CabinLetter'].fillna("?")
X['Pclass'] = X['Pclass'].astype(str)
X['SibSp'] = X['SibSp'].astype(str)
X['Parch'] = X['Parch'].astype(str)
X = X.dropna()
X

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,CabinLetter
0,3,male,22.0,1,0,7.2500,S,?
1,1,female,38.0,1,0,71.2833,C,C
2,3,female,26.0,0,0,7.9250,S,?
3,1,female,35.0,1,0,53.1000,S,C
4,3,male,35.0,0,0,8.0500,S,?
...,...,...,...,...,...,...,...,...
885,3,female,39.0,0,5,29.1250,Q,?
886,2,male,27.0,0,0,13.0000,S,?
887,1,female,19.0,0,0,30.0000,S,B
889,1,male,26.0,0,0,30.0000,C,C


In [5]:
X.dtypes

Pclass          object
Sex             object
Age            float64
SibSp           object
Parch           object
Fare           float64
Embarked        object
CabinLetter     object
dtype: object

We will first implement ID3 before we move towards C4.5. This means we cannot handle numeric data such as ``Age`` and ``Fare``. We will bin these into 20 categories. I picked 20 after trying a few different values. At this point, I do not know if it is a good selection or bad. This is part of the reason we will switch to C4.5. 

In [6]:
X2 = X.copy()
X2['Age'] = pd.cut(X2['Age'],bins=20).astype(str) # bin Age up
X2['Age'].value_counts()

(20.315, 24.294]    98
(24.294, 28.273]    85
(28.273, 32.252]    84
(16.336, 20.315]    79
(32.252, 36.231]    73
(36.231, 40.21]     44
(0.34, 4.399]       40
(44.189, 48.168]    35
(40.21, 44.189]     35
(12.357, 16.336]    31
(48.168, 52.147]    29
(52.147, 56.126]    16
(8.378, 12.357]     15
(4.399, 8.378]      14
(56.126, 60.105]    13
(60.105, 64.084]    10
(68.063, 72.042]     5
(64.084, 68.063]     4
(76.021, 80.0]       1
(72.042, 76.021]     1
Name: Age, dtype: int64

In [7]:
X2['Fare'] = pd.cut(X2['Fare'],bins=20).astype(str) # bin Age up
X2['Fare'].value_counts()

(-0.512, 25.616]      424
(25.616, 51.233]      153
(51.233, 76.849]       53
(76.849, 102.466]      34
(102.466, 128.082]     14
(128.082, 153.699]     14
(204.932, 230.548]      7
(256.165, 281.781]      6
(486.713, 512.329]      3
(230.548, 256.165]      2
(153.699, 179.315]      2
Name: Fare, dtype: int64

In [8]:
t = titanic_df.loc[X2.index,'Survived']
t

0      0
1      1
2      1
3      1
4      0
      ..
885    0
886    0
887    1
889    1
890    0
Name: Survived, Length: 712, dtype: int64

#### Exercise 1
Construct a function called ``entropy`` that calculates the entropy of a set (Pandas Series Object)

In [9]:
e1 = Lab4_helper.entropy(t)
e2 = Lab4_helper.entropy(X2['CabinLetter'])
e1,e2

(0.9735190023846809, 1.4723461729538008)

#### Exercise 2
Write a function called ``gain`` that calculates the information gain after splitting with a specific variable (Equation 12.2 from Marsland).

In [10]:
g1 = Lab4_helper.gain(t,X2['Sex'])
g2 = Lab4_helper.gain(t,X2['Pclass'])
g3 = Lab4_helper.gain(t,X2['Age'])
g1,g2,g3

(0.21410831283572307, 0.09400998456880616, 0.03712337530803511)

#### Exercise 3
C4.5 actually uses the gain ratio which is defined as the information gain "normalized" (divided) by the entropy before the split. You have written everything you need here. Just put it together.

In [11]:
gr1 = Lab4_helper.gain_ratio(t,X2['Sex'])
gr2 = Lab4_helper.gain_ratio(t,X2['Pclass'])
gr3 = Lab4_helper.gain_ratio(t,X2['Age'])
gr1,gr2,gr3

(0.21993234062330022, 0.09656717982753726, 0.03813317995550127)

#### Exercise 4
Define a function called ``select_split`` that chooses the column to place in the decision tree. This function returns the column name and the gain ratio for this column.

In [12]:
col,gain_ratio = Lab4_helper.select_split(X2,t)
col,gain_ratio

('Sex', 0.21993234062330022)

#### Exercise 5
Now put it all together and construct a function called ``make_tree`` that returns a tree in the format shown below. This function is a recursive function. Think carefully about how to debug recursion (i.e., grab yourself a debugger such as https://docs.python.org/3/library/pdb.html). Think carefully the base cases. 

In [13]:
tree = Lab4_helper.make_tree(X2,t)
Lab4_helper.print_tree(tree)

{
    "Sex": {
        "female": {
            "Pclass": {
                "1": {
                    "Age": {
                        "(0.34, 4.399]": 0,
                        "(12.357, 16.336]": 1,
                        "(16.336, 20.315]": 1,
                        "(20.315, 24.294]": 1,
                        "(24.294, 28.273]": {
                            "SibSp": {
                                "0": 1,
                                "1": 0
                            }
                        },
                        "(28.273, 32.252]": 1,
                        "(32.252, 36.231]": 1,
                        "(36.231, 40.21]": 1,
                        "(40.21, 44.189]": 1,
                        "(44.189, 48.168]": 1,
                        "(48.168, 52.147]": {
                            "CabinLetter": {
                                "B": 1,
                                "C": 0,
                                "D": 1
                            }
          

#### Exercise 6
Create a recrusive function called ``generate_rules`` that returns an array of the rules from a tree. A rule is the form of:
```python
[('Sex', 'male'),
 ('Age', '(20.315, 24.294]'),
 ('Embarked', 'S'),
 ('Pclass', 3),
 ('SibSp', 1),
 0]
```
A single rule has a type of list. The last element in the list is the prediction, which is Survived=0 in this example. The tuples that preceed the last element are the conditions. Put another way, the above rule is equivalent to:
```python
if Sex == 'male' and Age == '(20.315, 24.294]' and Embarked == 'S' and Pclass == 3 and SibSp == 1:
    predicted_value = 0
```

In [14]:
rules = Lab4_helper.generate_rules(tree)
rules[:5] # the first 5 rules

[[('Sex', 'male'),
  ('Age', '(20.315, 24.294]'),
  ('Embarked', 'S'),
  ('Pclass', '3'),
  ('SibSp', '1'),
  0],
 [('Sex', 'male'),
  ('Age', '(20.315, 24.294]'),
  ('Embarked', 'S'),
  ('Pclass', '3'),
  ('SibSp', '0'),
  0],
 [('Sex', 'male'),
  ('Age', '(20.315, 24.294]'),
  ('Embarked', 'S'),
  ('Pclass', '3'),
  ('SibSp', '2'),
  0],
 [('Sex', 'male'),
  ('Age', '(20.315, 24.294]'),
  ('Embarked', 'S'),
  ('Pclass', '2'),
  0],
 [('Sex', 'male'),
  ('Age', '(20.315, 24.294]'),
  ('Embarked', 'S'),
  ('Pclass', '1'),
  0]]

In [15]:
rules

[[('Sex', 'male'),
  ('Age', '(20.315, 24.294]'),
  ('Embarked', 'S'),
  ('Pclass', '3'),
  ('SibSp', '1'),
  0],
 [('Sex', 'male'),
  ('Age', '(20.315, 24.294]'),
  ('Embarked', 'S'),
  ('Pclass', '3'),
  ('SibSp', '0'),
  0],
 [('Sex', 'male'),
  ('Age', '(20.315, 24.294]'),
  ('Embarked', 'S'),
  ('Pclass', '3'),
  ('SibSp', '2'),
  0],
 [('Sex', 'male'),
  ('Age', '(20.315, 24.294]'),
  ('Embarked', 'S'),
  ('Pclass', '2'),
  0],
 [('Sex', 'male'),
  ('Age', '(20.315, 24.294]'),
  ('Embarked', 'S'),
  ('Pclass', '1'),
  0],
 [('Sex', 'male'),
  ('Age', '(20.315, 24.294]'),
  ('Embarked', 'C'),
  ('Fare', '(-0.512, 25.616]'),
  ('Pclass', '3'),
  0],
 [('Sex', 'male'),
  ('Age', '(20.315, 24.294]'),
  ('Embarked', 'C'),
  ('Fare', '(-0.512, 25.616]'),
  ('Pclass', '2'),
  0],
 [('Sex', 'male'),
  ('Age', '(20.315, 24.294]'),
  ('Embarked', 'C'),
  ('Fare', '(51.233, 76.849]'),
  1],
 [('Sex', 'male'),
  ('Age', '(20.315, 24.294]'),
  ('Embarked', 'C'),
  ('Fare', '(230.548, 256.165]

#### Exercise 7
Create an improved function to create a tree called ``make_tree2``. This function is a recursive function. This function must add support for numeric columns, and it must incorporate a parameter that battles overfitting called ``min_split_count``. Minimum split count is incorporated as an additional base case. To implement, check to see if you have at least min_split_count items (i.e., num_elements >= min_split_count to split). The biggest change comes with the addition of numeric columns (Age and Fare in their original format). Please refer to the Marsland textbook for details on handling numeric values. In short, you try all possible locations to divide a numeric variable. For example, if your column has the values:
```
values = [1,3,2,5]
sorted_values = [1,2,3,5]
possible_splits = [<1.5,<2.5,<4]
```
Please make sure you denote your splits like I am doing above and how they are printed below.

In [16]:
tree2 = Lab4_helper.make_tree2(X,t,min_split_count=20)
Lab4_helper.print_tree(tree2)

{
    "Sex": {
        "female": {
            "Pclass": {
                "1": {
                    "CabinLetter": {
                        "?": 1,
                        "A": 1,
                        "B": 1,
                        "C": {
                            "Age<9.50": {
                                "False": {
                                    "Fare<39.11": {
                                        "False": {
                                            "Parch": {
                                                "0": 1,
                                                "1": 1,
                                                "2": 1
                                            }
                                        },
                                        "True": 0
                                    }
                                },
                                "True": 0
                            }
                        },
                        "D": 1,
 

#### Exercise 8
So how are we doing? We can put everything together and evaluate our solutions.

Create a function to make predictions called ``make_prediction``. Then use your Lab3_helper solutions to do some evaluations.

In [17]:
default = 0
from sklearn.model_selection import train_test_split

X2_train, X2_test, t_train, t_test = train_test_split(X2, t, test_size=0.3, random_state = 0)
X_train, X_test = X.loc[X2_train.index], X.loc[X2_test.index]

tree_id3 = Lab4_helper.make_tree(X2_train,t_train)
rules_id3 = Lab4_helper.generate_rules(tree_id3)
tree_c45 = Lab4_helper.make_tree2(X_train,t_train)
rules_c45 = Lab4_helper.generate_rules(tree_c45)

y_id3 = X2_test.apply(lambda x: Lab4_helper.make_prediction(rules_id3,x,default),axis=1)
y_c45 = X_test.apply(lambda x: Lab4_helper.make_prediction(rules_c45,x,default),axis=1)

In [18]:
import Lab3_helper

In [19]:
# Evaluate the id3
cm_id3 = Lab3_helper.confusion_matrix(t_test,y_id3,labels=[0,1])
stats_id3 = Lab3_helper.evaluation(cm_id3,positive_class=1)
stats_id3

{'accuracy': 0.7523364485981309,
 'sensitivity/recall': 0.5955056179775281,
 'specificity': 0.864,
 'precision': 0.7571428571428571,
 'F1': 0.6666666666666667}

In [20]:
# Evaluate the c45
cm_c45 = Lab3_helper.confusion_matrix(t_test,y_c45,labels=[0,1])
stats_c45 = Lab3_helper.evaluation(cm_c45,positive_class=1)
stats_c45

{'accuracy': 0.7616822429906542,
 'sensitivity/recall': 0.7078651685393258,
 'specificity': 0.8,
 'precision': 0.7159090909090909,
 'F1': 0.711864406779661}

In [21]:
source = pd.DataFrame.from_records([stats_id3,stats_c45])
source['Method'] = ['ID3','C4.5']
source

,accuracy,sensitivity/recall,specificity,precision,F1,Method
0,0.752336,0.595506,0.864,0.757143,0.666667,ID3
1,0.761682,0.707865,0.800,0.715909,0.711864,C4.5


**Problem 1:** How do the two algorithms compare for this dataset?

Your answer here

**Problem 2:** Is this a robust experiment? How would you make it more robust? i.e., what are the flaws with what we did?

Your answer here

**Problem 3:** Repeat this experiment with min_split_count = 10, 20, 40, 80. How do the results change for C4.5?

Your answer here

In [22]:
# Good job!
# Don't forget to push with ./submit.sh

#### Having trouble with the test cases and the autograder?

You can always load up the answers for the autograder. The autograder runs your code and compares your answer to the expected answer. I manually review your code, so there is no need to hide this from you.

```python
import joblib
answers = joblib.load(f"{home}/csc-466-student/tests/answers_Lab4.joblib")
answers.keys()
```